In [2]:
'''import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard
import time
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
#from keras.models import Model
#from keras.layers import Input, Dense'''
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku 
import numpy as np

tokenizer = Tokenizer()
tf.__version__

ModuleNotFoundError: No module named 'tensorflow.python'

In [2]:
# data I/O
data = open('input_whatsapp.txt', 'r').read() #simple text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))

data has 909889 characters, 394 unique.


In [3]:
def dataset_preparation(data):

	# basic cleanup
	corpus = data.lower().split("\n")

	# tokenization	
	tokenizer.fit_on_texts(corpus)
	total_words = len(tokenizer.word_index) + 1

	# create input sequences using list of tokens
	input_sequences = []
	for line in corpus:
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(1, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)

	# pad sequences 
	max_sequence_len = max([len(x) for x in input_sequences])
	input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

	# create predictors and label
	predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
	label = ku.to_categorical(label, num_classes=total_words)

	return predictors, label, max_sequence_len, total_words

In [4]:
def create_model(predictors, label, max_sequence_len, total_words):
	
	model = Sequential()
	model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
	model.add(LSTM(150, return_sequences = True))
	model.add(LSTM(100))
	model.add(Dense(total_words, activation='softmax'))

	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

	print (model.summary())
	return model 

In [5]:
def generate_text(seed_text, next_words, max_sequence_len):
	for _ in range(next_words):
		token_list = tokenizer.texts_to_sequences([seed_text])[0]
		token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
		predicted = model.predict_classes(token_list, verbose=0)
		
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == predicted:
				output_word = word
				break
		seed_text += " " + output_word
	return seed_text

In [6]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)
print(predictors)
print(label)
print(max_sequence_len)
print(total_words)

[[    0     0     0 ...     0     0  5643]
 [    0     0     0 ...     0  5643   468]
 [    0     0     0 ...  5643   468   403]
 ...
 [    0     0     0 ... 14599  1014     2]
 [    0     0     0 ...  1014     2   114]
 [    0     0     0 ...     2   114    75]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
196
14601


In [7]:
model = create_model(predictors, label, max_sequence_len, total_words)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 195, 10)           146010    
_________________________________________________________________
lstm (LSTM)                  (None, 195, 150)          96600     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               100400    
_________________________________________________________________
dense (Dense)                (None, 14601)             1474701   
Total params: 1,817,711
Trainable params: 1,817,711
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
model.load_weights('weights/model_weights_5_0')

In [9]:
model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop], validation_split=0.1)

W0624 18:39:20.459333 140736110142336 deprecation.py:323] From /nfs/2018/j/jcruz-y-/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 119610 samples, validate on 13290 samples
Epoch 1/100
119610/119610 [==============================] - 1824s 15ms/sample - loss: 5.0309 - accuracy: 0.2288 - val_loss: 5.7793 - val_accuracy: 0.2647
Epoch 2/100
119610/119610 [==============================] - 1835s 15ms/sample - loss: 4.9442 - accuracy: 0.2309 - val_loss: 5.7927 - val_accuracy: 0.2662
Epoch 3/100
119610/119610 [==============================] - 1263s 11ms/sample - loss: 4.8624 - accuracy: 0.2340 - val_loss: 5.8350 - val_accuracy: 0.2570
Epoch 4/100
119610/119610 [==============================] - 1154s 10ms/sample - loss: 4.7839 - accuracy: 0.2376 - val_loss: 5.8180 - val_accuracy: 0.2701
Epoch 5/100
119610/119610 [==============================] - 1153s 10ms/sample - loss: 4.7163 - accuracy: 0.2389 - val_loss: 5.8184 - val_accuracy: 0.2705
Epoch 6/100
119610/119610 [==============================] - 1159s 10ms/sample - loss: 4.6518 - accuracy: 0.2414 - val_loss: 5.8218 - val_accuracy: 0.2702


In [48]:
print (generate_text("Fer Ripoll Lilo Domit", 100, max_sequence_len))

Fer Ripoll Lilo Domit media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted media omitted


In [49]:
model.save_weights("weights/model_weights_5_0", overwrite=True, save_format='tf')

In [165]:
def batch_gen(seq_length, input_dim, data):
    j = 0
    batches = data.shape[0] // seq_length
    x = np.zeros((batches, seq_length, input_dim))
    for batch in range(batches):
        for i in range(seq_length):
            x[batch, i, :] = data.iloc[j, :]
            j += 1
    return (x)

In [166]:
len(x_train.iloc[:, 1])
#Xtrain.shape[0]

3082

In [188]:
# 3D tensor with shape (batch_size, timesteps, input_dim).
x = batch_gen(512, 1, x_train)
x.shape
y = batch_gen(512, 1, y_train)
y.shape

(6, 512, 63)

In [189]:
inputs = tf.keras.layers.Input(shape=(None, 8))

In [190]:
lstm = tf.keras.layers.LSTM(512,
                            activation='tanh',
                            recurrent_activation='hard_sigmoid',
                            use_bias=True,
                            kernel_initializer='glorot_uniform',
                            recurrent_initializer='orthogonal',
                            bias_initializer='zeros',
                            unit_forget_bias=True, 
                            kernel_regularizer=None,
                            recurrent_regularizer=None,
                            bias_regularizer=None,
                            activity_regularizer=None,
                            kernel_constraint=None,
                            recurrent_constraint=None,
                            bias_constraint=None,
                            dropout=0.5, 
                            recurrent_dropout=0.5,
                            implementation=1, 
                            return_sequences=True, 
                            return_state=False, 
                            go_backwards=False, 
                            stateful=False,
                            unroll=False)(inputs)
lstm2 = tf.keras.layers.LSTM(512,
                            dropout=0.5, 
                            recurrent_dropout=0.5,
                            return_sequences=True)(lstm)
predictions = tf.keras.layers.Dense(63, activation='relu')(lstm2)
#predictions = tf.keras.layers.TimeDistributed(dense)(dense)

In [191]:
model = tf.keras.models.Model(inputs=inputs, outputs=predictions)

In [192]:
model.compile(optimizer='rmsprop', loss='mean_squared_error')

In [193]:
print(model.summary())

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, None, 8)]         0         
_________________________________________________________________
lstm_23 (LSTM)               (None, None, 512)         1067008   
_________________________________________________________________
lstm_24 (LSTM)               (None, None, 512)         2099200   
_________________________________________________________________
dense_16 (Dense)             (None, None, 63)          32319     
Total params: 3,198,527
Trainable params: 3,198,527
Non-trainable params: 0
_________________________________________________________________
None


In [194]:
history = model.fit(x,
                   y,
                   epochs=10000)

Train on 6 samples
Epoch 1/10000
6/6 [==============================] - 10s 2s/sample - loss: 1523.6422
Epoch 2/10000
6/6 [==============================] - 6s 1s/sample - loss: 1516.0851
Epoch 3/10000
6/6 [==============================] - 7s 1s/sample - loss: 1506.9043
Epoch 4/10000
6/6 [==============================] - 8s 1s/sample - loss: 1484.7946
Epoch 5/10000
6/6 [==============================] - 10s 2s/sample - loss: 1454.6826
Epoch 6/10000
6/6 [==============================] - 7s 1s/sample - loss: 1428.4946
Epoch 7/10000
6/6 [==============================] - 7s 1s/sample - loss: 1408.1478
Epoch 8/10000
6/6 [==============================] - 9s 1s/sample - loss: 1394.6260
Epoch 9/10000
6/6 [==============================] - 8s 1s/sample - loss: 1384.7266
Epoch 10/10000
6/6 [==============================] - 8s 1s/sample - loss: 1376.0287
Epoch 11/10000
6/6 [==============================] - 8s 1s/sample - loss: 1369.3052
Epoch 12/10000
6/6 [==============================] -

6/6 [==============================] - 6s 1s/sample - loss: 1305.9996
Epoch 98/10000
6/6 [==============================] - 6s 1s/sample - loss: 1305.8763
Epoch 99/10000
6/6 [==============================] - 6s 1s/sample - loss: 1306.2821
Epoch 100/10000
6/6 [==============================] - 6s 1s/sample - loss: 1305.6476
Epoch 101/10000
6/6 [==============================] - 6s 1s/sample - loss: 1306.8558
Epoch 102/10000
6/6 [==============================] - 6s 1s/sample - loss: 1306.4430
Epoch 103/10000
6/6 [==============================] - 6s 1s/sample - loss: 1305.3318
Epoch 104/10000
6/6 [==============================] - 6s 1s/sample - loss: 1305.9449
Epoch 105/10000
6/6 [==============================] - 8s 1s/sample - loss: 1305.2540
Epoch 106/10000
6/6 [==============================] - 12s 2s/sample - loss: 1305.2733
Epoch 107/10000
6/6 [==============================] - 12s 2s/sample - loss: 1305.7115
Epoch 108/10000
6/6 [==============================] - 9s 1s/sample - 

KeyboardInterrupt: 